3) Se tiene un RDD con info de vvuelos programados con la foma (numero de vuelo, codigo de aerolinea, codigo de aeropuerto de salida, codigo de aeropuerto de llegada, fecha de salida AAAAMMDD, hora de salida HH:MM, fecha de llega AAAAMMDD, hora de llegada HH:MM). A su vez, se cuenta con el registro actualizado del estado dee los vuelos que fueron ocurriendo, con formato (numero de vuelo, aerolinea, fecha de salida, hora de salida, fecha de llegada, hora de llegada, estado). En base al estado, podria contar con alggun dato en blanco, por ejemplo si el vuelo fue cancelado no tendra info de fechas y horas, si el vuelvo se encuentra aun en curso, no contendra info de llegada. Se pide resolver con PySpark:
    **A. Cual es el aeropuerto con mayor transito.
      B. Cual es la aerolinea con mayor cantidad de vuelos.
      C. Cual es la aerolinea con mayor cantidad de cancelaciones.
      D. Cual es el vuelo (num. de vuelo+ fecha) con mayor retraso en          horario de salida.
      E. Cual es el vuelo (num.de vuelo+fecha) con mayor retraso en
         horario de llegada.
      F. Cual es la aerolinea mas puntual.
      G. Cual es el aeropuerto que registra mayor desviacion con                respecto a los horarios coordinados.**

In [1]:
import random
from fechasRandom import *
from datetime import *

vuelos = [(1,random.randint(1,3),'AEP','MDQ','20200320','06:15','20200320','07:05'),
          (2,random.randint(1,3),'AEP','CPC','20200320','12:00','20200320','14:15'),
          (3,random.randint(1,3),'BRC','EZE','20200320','17:15','20200320','19:30'),
          (4,random.randint(1,3),'AEP','IGR','20200320','21:00','20200320','22:40'),
          (5,random.randint(1,3),'MDQ','IGR','20200321','06:00','20200321','08:00'),
          (6,random.randint(1,3),'IGR','USH','20200321','21:00','20200322','04:20'),
          (7,random.randint(1,3),'USH','AEP','20200322','15:15','20200322','18:20'),
          (8,random.randint(1,3),'AEP','RGA','20200322','17:00','20200322','20:15'),
          (9,random.randint(1,3),'BHI','COR','20200323','09:10','20200323','14:00'),
          (10,random.randint(1,3),'NQN','BRC','20200323','14:45','20200323','15:05')]

estados = [(1,vuelos[0][1],vuelos[0][4],vuelos[0][5],vuelos[0][6],vuelos[0][7],'Finalizado'),
           (2,vuelos[1][1],vuelos[1][4],vuelos[1][5],vuelos[1][6],'14:28','Finalizado'),
           (3,vuelos[2][1],vuelos[2][4],vuelos[2][5],None,None,'En curso'),
           (4,vuelos[3][1],None,None,None,None,'Cancelado'),
           (5,vuelos[4][1],vuelos[4][4],'06:23',vuelos[4][6],'08:30','Finalizado'),
           (6,vuelos[5][1],None,None,None,None,'Cancelado'),
           (7,vuelos[6][1],vuelos[6][4],vuelos[6][5],vuelos[6][6],vuelos[6][7],'Finalizado'),
           (8,vuelos[7][1],vuelos[7][4],'17:05',None,None,'En curso'),
           (9,vuelos[8][1],None,None,None,None,'Cancelado'),
           (10,vuelos[9][1],vuelos[9][4],vuelos[9][5],vuelos[9][6],vuelos[9][7],'Finalizado')]

In [2]:
vuelosRDD = sc.parallelize(vuelos)
vuelosRDD.collect()

[(1, 2, 'AEP', 'MDQ', '20200320', '06:15', '20200320', '07:05'),
 (2, 1, 'AEP', 'CPC', '20200320', '12:00', '20200320', '14:15'),
 (3, 3, 'BRC', 'EZE', '20200320', '17:15', '20200320', '19:30'),
 (4, 3, 'AEP', 'IGR', '20200320', '21:00', '20200320', '22:40'),
 (5, 3, 'MDQ', 'IGR', '20200321', '06:00', '20200321', '08:00'),
 (6, 1, 'IGR', 'USH', '20200321', '21:00', '20200322', '04:20'),
 (7, 3, 'USH', 'AEP', '20200322', '15:15', '20200322', '18:20'),
 (8, 2, 'AEP', 'RGA', '20200322', '17:00', '20200322', '20:15'),
 (9, 3, 'BHI', 'COR', '20200323', '09:10', '20200323', '14:00'),
 (10, 2, 'NQN', 'BRC', '20200323', '14:45', '20200323', '15:05')]

In [3]:
estadosRDD = sc.parallelize(estados)
estadosRDD.collect()

[(1, 2, '20200320', '06:15', '20200320', '07:05', 'Finalizado'),
 (2, 1, '20200320', '12:00', '20200320', '14:28', 'Finalizado'),
 (3, 3, '20200320', '17:15', None, None, 'En curso'),
 (4, 3, None, None, None, None, 'Cancelado'),
 (5, 3, '20200321', '06:23', '20200321', '08:30', 'Finalizado'),
 (6, 1, None, None, None, None, 'Cancelado'),
 (7, 3, '20200322', '15:15', '20200322', '18:20', 'Finalizado'),
 (8, 2, '20200322', '17:05', None, None, 'En curso'),
 (9, 3, None, None, None, None, 'Cancelado'),
 (10, 2, '20200323', '14:45', '20200323', '15:05', 'Finalizado')]

**A. Cual es el aeropuerto con mayor transito**

In [4]:
#tengo que ver, tanto en aeropuertos de llegada como de salida, cuantas veces se repiten
#y quedarme con el que mas se repite

#(aeropSalida, cantidadApariciones)
aeropSalidaRDD = vuelosRDD.map(lambda x: (x[2],1))
aeropSalidaRDD = aeropSalidaRDD.reduceByKey(lambda x,y: x+y)

#(aeropLlegada, cantidadApariciones)
aeropLlegadaRDD = vuelosRDD.map(lambda x: (x[3],1))
aeropLlegadaRDD = aeropLlegadaRDD.reduceByKey(lambda x,y: x+y)

#junto los RDD -> (aerop, (cantidadSalida,cantidadLlegada))
aeropuertosRDD = aeropSalidaRDD.join(aeropLlegadaRDD)

#(aerop, cantidadSalida + cantidadLlegada)
aeropuertosRDD = aeropuertosRDD.map(lambda x: (x[0], x[1][0]+x[1][1]))

#me quedo con el mayor
aeropuertosRDD = aeropuertosRDD.takeOrdered(aeropuertosRDD.count(),lambda x: x[1])
print("El aeropuerto con mayor transito es '{}' con {} vuelos".format(aeropuertosRDD[-1][0], aeropuertosRDD[-1][1]))

El aeropuerto con mayor transito es 'AEP' con 5 vuelos


**B. Cual es la aerolinea con mayor cantidad de vuelos.**

In [5]:
aerolineasRDD = vuelosRDD.map(lambda x: (x[1],1))
aerolineasRDD = aerolineasRDD.reduceByKey(lambda x,y: x+y)
aerolineasRDD = aerolineasRDD.takeOrdered(aerolineasRDD.count(),lambda x:x[1])
print("La aerolinea con mayor cantidad de vuelos es la N° {} con {} vuelos".format(aerolineasRDD[-1][0],aerolineasRDD[-1][1]))

La aerolinea con mayor cantidad de vuelos es la N° 3 con 5 vuelos


**C. Cual es la aerolinea con mayor cantidad de cancelaciones.**

In [6]:
#me quedo con los vuelos que tengan estado cancelado
cancelacionesRDD = estadosRDD.filter(lambda x: x[6] == 'Cancelado')
cancelacionesRDD = cancelacionesRDD.map(lambda x: (x[1],1))
cancelacionesRDD = cancelacionesRDD.reduceByKey(lambda x,y: x+y)
cancelacionesRDD = cancelacionesRDD.takeOrdered(cancelacionesRDD.count(),lambda x: x[1])
print("La aerolinea con mas cancelaciones es la N° {} con {} cancelaciones".format(cancelacionesRDD[-1][0],cancelacionesRDD[-1][1]))

La aerolinea con mas cancelaciones es la N° 3 con 2 cancelaciones


**D. Cual es el vuelo (num. de vuelo+ fecha) con mayor retraso en horario de salida.**

In [35]:
horarioTeoricoRDD = vuelosRDD.map(lambda x: ((x[0],x[4]),datetime.strptime(x[5],'%H:%M')))
horarioRealRDD = estadosRDD.filter(lambda x: x[6] != 'Cancelado')
horarioRealRDD = horarioRealRDD.map(lambda x: ((x[0],x[2]),datetime.strptime(x[3],'%H:%M')))

#En horarioTeorico tengo TODOS los vuelos, incluso los cancelados
#En horarioReal tengo los vuelos que finalizaron o que estan en curso
#si hago join, voy a tener un RDD nuevo con los vuelos finalizados o en curso, y se borran los cancelados
#((vuelo,fechaSalida),(fechaTeorica,fechaReal))
horariosRDD = horarioTeoricoRDD.join(horarioRealRDD)
#(vuelo,fechaSalida, segundosDiferenciaHorarios) (horarioReal-horarioTeorico)
horariosRDD = horariosRDD.map(lambda x: (x[0][0],x[0][1],(x[1][1]-x[1][0])))
horariosRDD = horariosRDD.takeOrdered(horariosRDD.count(), lambda x: x[2])
#formatear
print("El vuelo con mayor retraso en horario de salida es el vuelo N°{}, con fecha {} y un retraso de {} horas".format(horariosRDD[-1][0],horariosRDD[-1][1],horariosRDD[-1][2]))

El vuelvo con mayor retraso en horario de salida es el vuelo N°5, con fecha 20200321 y un retraso de 0:23:00 horas


**E. Cual es el vuelo (num.de vuelo+fecha) con mayor retraso en horario de llegada**

In [50]:
#similar al anterior
#a horarioRealRDD le saco los vuelos en curso para quedarme solo con los finalizados
horarioLlegadaTeoricoRDD = vuelosRDD.map(lambda x: ((x[0],x[6]),datetime.strptime(x[7],'%H:%M')))
horarioLlegadaRealRDD = estadosRDD.filter(lambda x: (x[6] != 'Cancelado') and (x[6] != 'En curso'))
horarioLlegadaRealRDD = horarioLlegadaRealRDD.map(lambda x: ((x[0],x[4]),datetime.strptime(x[5],'%H:%M')))

horariosRDD2 = horarioLlegadaTeoricoRDD.join(horarioLlegadaRealRDD)

horariosRDD2 = horariosRDD2.map(lambda x: (x[0][0],x[0][1],(x[1][1]-x[1][0])))
horariosRDD2 = horariosRDD2.takeOrdered(horariosRDD2.count(), lambda x: x[2])
#format
print("El vuelo con mayor retraso en horario de llegada es el vuelo N°{}, con fecha {} y un retraso de {} horas".format(horariosRDD2[-1][0],horariosRDD2[-1][1],horariosRDD2[-1][2]))

El vuelo con mayor retraso en horario de llegada es el vuelo N°5, con fecha 20200321 y un retraso de 0:30:00 horas


**F. Cual es la aerolinea mas puntual**

In [71]:
#(vuelo, fechaLlegada,timedelta)
horariosLlegadaRDD = horarioLlegadaTeoricoRDD.join(horarioLlegadaRealRDD)
horariosLlegadaRDD = horariosLlegadaRDD.map(lambda x: (x[0][0],x[0][1],(x[1][1]-x[1][0])))
#(vuelo,timedelta)
horariosLlegadaRDD = horariosLlegadaRDD.map(lambda x: (x[0],x[2]))

#(vuelo,fechaSalida,timedelta)
horariosSalidaRDD = horarioTeoricoRDD.join(horarioRealRDD)
horariosSalidaRDD = horariosSalidaRDD.map(lambda x: (x[0][0],x[0][1],(x[1][1]-x[1][0])))
#(vuelo, timeldeta)
horariosSalidaRDD = horariosSalidaRDD.map(lambda x: (x[0],x[2]))

#(vuelo, aerolinea)
aerolineasYVuelosRDD = estadosRDD.map(lambda x: (x[0],x[1]))

#joineo por aerolinea y vuelo
aerolineasDeltaLlegadaRDD = aerolineasYVuelosRDD.join(horariosLlegadaRDD)
#(aerolinea,delta)
aerolineasDeltaLlegadaRDD = aerolineasDeltaLlegadaRDD.map(lambda x: (x[1][0],x[1][1]))
#(aerolinea, suma de los retrasos de llegada)
aerolineasDeltaLlegadaRDD = aerolineasDeltaLlegadaRDD.reduceByKey(lambda x,y: x+y)

aerolineasDeltaSalidaRDD = aerolineasYVuelosRDD.join(horariosSalidaRDD)
aerolineasDeltaSalidaRDD = aerolineasDeltaSalidaRDD.map(lambda x: (x[1][0],x[1][1]))
aerolineasDeltaSalidaRDD = aerolineasDeltaSalidaRDD.reduceByKey(lambda x,y: x+y)

aerolineasDeltaRDD = aerolineasDeltaSalidaRDD.union(aerolineasDeltaLlegadaRDD)
aerolineasDeltaRDD = aerolineasDeltaRDD.reduceByKey(lambda x,y: x+y)
aerolineasDeltaRDD = aerolineasDeltaRDD.takeOrdered(aerolineasDeltaRDD.count(),lambda x: x[1].total_seconds())
aerolineasDeltaRDD

print("La aerolinea mas puntual es la N°{}".format(aerolineasDeltaRDD[0][0]))

La aerolinea mas puntual es la N°2
